# 更新Fields DB
* 所有資料匯入DB之前要改成 
* index: datetime 
* columns: 股票代號、商品名稱 str
* 日期要擁有一個columns

In [1]:
import pymongo
import pandas as pd
import time
import datetime
import xlwings as xw
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數


In [2]:
client = pymongo.MongoClient()
db = client['Fields']

In [3]:
# 季資料日期轉換
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)


wb1 = xw.Book(r"C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm")
func_all = wb1.macro("CM全部更新")
func_all()
time.sleep(5)
wb2 = xw.Book(r"C:\Users\ChastLai\Documents\Data\每日更新DB\CMoney單日更新.xlsm")
func_ = wb2.macro("CM更新")
func_('一日')


wb1.save()
wb2.save()
EPS_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='EPS_Q', header=4)
股票名稱轉換 = dict(zip(EPS_Q['股票代號'].astype(str), EPS_Q['股票名稱']))
EPS_Q = EPS_Q.drop('股票名稱', axis=1).set_index('股票代號')
EPS_Q.columns = [s.split('每')[0] for s in EPS_Q.columns]
EPS_Q.columns = change_Q(EPS_Q.columns)
EPS_Q.columns = pd.to_datetime(EPS_Q.columns)
EPS_Q = EPS_Q.T.dropna(how='all', axis=0)

預估EPS_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='預估EPS_Q', header=4)
預估EPS_Q = 預估EPS_Q.drop('股票名稱', axis=1).set_index('股票代號')
預估EPS_Q.columns = [s.split('稅')[0] for s in 預估EPS_Q.columns]
預估EPS_Q.columns = change_Q(預估EPS_Q.columns)
預估EPS_Q.columns = pd.to_datetime(預估EPS_Q.columns)
預估EPS_Q = 預估EPS_Q.T.dropna(how='all', axis=0)

# 這個為年單位
營業收入淨額_Y = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='營業收入淨額_Y', header=4)
營業收入淨額_Y = 營業收入淨額_Y.drop('股票名稱', axis=1).set_index('股票代號')
# 年資料日期轉換
營業收入淨額_Y.columns = [str(int(s.split('營')[0]) + 1) + '0331' for s in 營業收入淨額_Y.columns]
營業收入淨額_Y.columns = pd.to_datetime(營業收入淨額_Y.columns)
營業收入淨額_Y = 營業收入淨額_Y.T.dropna(how='all', axis=0)

營業收入淨額_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='營業收入淨額_Q', header=4)
營業收入淨額_Q = 營業收入淨額_Q.drop('股票名稱', axis=1).set_index('股票代號')
# 月資料日期轉換
營業收入淨額_Q.columns = [s.split('營')[0] for s in 營業收入淨額_Q.columns]
營業收入淨額_Q.columns = change_Q(營業收入淨額_Q.columns)
營業收入淨額_Q.columns = pd.to_datetime(營業收入淨額_Q.columns)
營業收入淨額_Q = 營業收入淨額_Q.T.dropna(how='all', axis=0)

EBITDA_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='EBITDA_Q', header=4)
EBITDA_Q = EBITDA_Q.drop('股票名稱', axis=1).set_index('股票代號')
# 月資料日期轉換
EBITDA_Q.columns = [s.split('E')[0] for s in EBITDA_Q.columns]
EBITDA_Q.columns = change_Q(EBITDA_Q.columns)
EBITDA_Q.columns = pd.to_datetime(EBITDA_Q.columns)
EBITDA_Q = EBITDA_Q.T.dropna(how='all', axis=0)

殖利率 = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='殖利率', header=4)
殖利率 = 殖利率.drop('股票名稱', axis=1).set_index('股票代號')
殖利率.columns = [s.split('殖')[0] for s in 殖利率.columns]
殖利率.columns = pd.to_datetime(殖利率.columns)
殖利率 = 殖利率.T.dropna(how='all', axis=0)

# 把股票代號改成字串
EPS_Q.columns = EPS_Q.columns.astype(str)
預估EPS_Q.columns = 預估EPS_Q.columns.astype(str)
殖利率.columns = 殖利率.columns.astype(str)
營業收入淨額_Y.columns = 營業收入淨額_Y.columns.astype(str)
營業收入淨額_Q.columns = 營業收入淨額_Q.columns.astype(str)
EBITDA_Q.columns = EBITDA_Q.columns.astype(str)
# 將日期也拉出來
EPS_Q['日期'] = EPS_Q.index
預估EPS_Q['日期'] = 預估EPS_Q.index
殖利率['日期'] = 殖利率.index
營業收入淨額_Y['日期'] = 營業收入淨額_Y.index
營業收入淨額_Q['日期'] = 營業收入淨額_Q.index
EBITDA_Q['日期'] = EBITDA_Q.index

# Q新增樣本
現金股利殖利率_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='現金股利殖利率_Q', header=4)
現金股利殖利率_Q = 現金股利殖利率_Q.drop('股票名稱', axis=1).set_index('股票代號')
# 月資料日期轉換
# 現金股利殖利率_Q.columns = [s.split('營')[0] for s in 現金股利殖利率_Q.columns]
現金股利殖利率_Q.columns = change_Q(現金股利殖利率_Q.columns)
現金股利殖利率_Q.columns = pd.to_datetime(現金股利殖利率_Q.columns)
現金股利殖利率_Q = 現金股利殖利率_Q.T.dropna(how='all', axis=0)
現金股利殖利率_Q.columns = 現金股利殖利率_Q.columns.astype(str)
現金股利殖利率_Q['日期'] = 現金股利殖利率_Q.index
update_mongo(現金股利殖利率_Q, '現金股利殖利率_Q')

槓桿比率_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='槓桿比率_Q', header=4)
槓桿比率_Q = 槓桿比率_Q.drop('股票名稱', axis=1).set_index('股票代號')
# 月資料日期轉換
# 現金股利殖利率_Q.columns = [s.split('營')[0] for s in 現金股利殖利率_Q.columns]
槓桿比率_Q.columns = change_Q(槓桿比率_Q.columns)
槓桿比率_Q.columns = pd.to_datetime(槓桿比率_Q.columns)
槓桿比率_Q = 槓桿比率_Q.T.dropna(how='all', axis=0)
槓桿比率_Q.columns = 槓桿比率_Q.columns.astype(str)
槓桿比率_Q['日期'] = 槓桿比率_Q.index
update_mongo(槓桿比率_Q, '槓桿比率_Q')

現金流量比率_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='現金流量比率_Q', header=4)
現金流量比率_Q = 現金流量比率_Q.drop('股票名稱', axis=1).set_index('股票代號')
# 月資料日期轉換
# 現金股利殖利率_Q.columns = [s.split('營')[0] for s in 現金股利殖利率_Q.columns]
現金流量比率_Q.columns = change_Q(現金流量比率_Q.columns)
現金流量比率_Q.columns = pd.to_datetime(現金流量比率_Q.columns)
現金流量比率_Q = 現金流量比率_Q.T.dropna(how='all', axis=0)
現金流量比率_Q.columns = 現金流量比率_Q.columns.astype(str)
現金流量比率_Q['日期'] = 現金流量比率_Q.index
update_mongo(現金流量比率_Q, '現金流量比率_Q')

# Y新增樣本
現金股利殖利率_Y = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='現金股利殖利率_Y', header=4)
現金股利殖利率_Y = 現金股利殖利率_Y.drop('股票名稱', axis=1).set_index('股票代號')
# 年資料日期轉換
現金股利殖利率_Y.columns = [str(int(s.split('現')[0]) + 1) + '0331' for s in 現金股利殖利率_Y.columns]
現金股利殖利率_Y.columns = pd.to_datetime(現金股利殖利率_Y.columns)
現金股利殖利率_Y = 現金股利殖利率_Y.T.dropna(how='all', axis=0)
現金股利殖利率_Y.columns = 現金股利殖利率_Y.columns.astype(str)
現金股利殖利率_Y['日期'] = 現金股利殖利率_Y.index
update_mongo(現金股利殖利率_Y, '現金股利殖利率_Y')

總資產週轉次數_Y = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='總資產週轉次數_Y', header=4)
總資產週轉次數_Y = 總資產週轉次數_Y.drop('股票名稱', axis=1).set_index('股票代號')
# 年資料日期轉換
總資產週轉次數_Y.columns = [str(int(s.split('總')[0]) + 1) + '0331' for s in 總資產週轉次數_Y.columns]
總資產週轉次數_Y.columns = pd.to_datetime(總資產週轉次數_Y.columns)
總資產週轉次數_Y = 總資產週轉次數_Y.T.dropna(how='all', axis=0)
總資產週轉次數_Y.columns = 總資產週轉次數_Y.columns.astype(str)
總資產週轉次數_Y['日期'] = 總資產週轉次數_Y.index
update_mongo(總資產週轉次數_Y, '總資產週轉次數_Y')

槓桿比率_Y = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='槓桿比率_Y', header=4)
槓桿比率_Y = 槓桿比率_Y.drop('股票名稱', axis=1).set_index('股票代號')
# 年資料日期轉換
槓桿比率_Y.columns = [str(int(s.split('槓')[0]) + 1) + '0331' for s in 槓桿比率_Y.columns]
槓桿比率_Y.columns = pd.to_datetime(槓桿比率_Y.columns)
槓桿比率_Y = 槓桿比率_Y.T.dropna(how='all', axis=0)
槓桿比率_Y.columns = 槓桿比率_Y.columns.astype(str)
槓桿比率_Y['日期'] = 槓桿比率_Y.index
update_mongo(槓桿比率_Y, '槓桿比率_Y')

現金流量比率_Y = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='現金流量比率_Y', header=4)
現金流量比率_Y = 現金流量比率_Y.drop('股票名稱', axis=1).set_index('股票代號')
# 年資料日期轉換
現金流量比率_Y.columns = [str(int(s.split('現')[0]) + 1) + '0331' for s in 現金流量比率_Y.columns]
現金流量比率_Y.columns = pd.to_datetime(現金流量比率_Y.columns)
現金流量比率_Y = 現金流量比率_Y.T.dropna(how='all', axis=0)
現金流量比率_Y.columns = 現金流量比率_Y.columns.astype(str)
現金流量比率_Y['日期'] = 現金流量比率_Y.index
update_mongo(現金流量比率_Y, '現金流量比率_Y')

In [14]:
現金流量比率_Q = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='現金流量比率_Q', header=4)
現金流量比率_Q = 現金流量比率_Q.drop('股票名稱', axis=1).set_index('股票代號')
# 月資料日期轉換
# 現金股利殖利率_Q.columns = [s.split('營')[0] for s in 現金股利殖利率_Q.columns]
現金流量比率_Q.columns = change_Q(現金流量比率_Q.columns)
現金流量比率_Q.columns = pd.to_datetime(現金流量比率_Q.columns)
現金流量比率_Q = 現金流量比率_Q.T.dropna(how='all', axis=0)
現金流量比率_Q.columns = 現金流量比率_Q.columns.astype(str)
現金流量比率_Q['日期'] = 現金流量比率_Q.index
update_mongo(現金流量比率_Q, '現金流量比率_Q')

現金流量比率_Y = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\價值股資料.xlsm', sheet_name='現金流量比率_Y', header=4)
現金流量比率_Y = 現金流量比率_Y.drop('股票名稱', axis=1).set_index('股票代號')
# 年資料日期轉換
現金流量比率_Y.columns = [str(int(s.split('現')[0]) + 1) + '0331' for s in 現金流量比率_Y.columns]
現金流量比率_Y.columns = pd.to_datetime(現金流量比率_Y.columns)
現金流量比率_Y = 現金流量比率_Y.T.dropna(how='all', axis=0)
現金流量比率_Y.columns = 現金流量比率_Y.columns.astype(str)
現金流量比率_Y['日期'] = 現金流量比率_Y.index
update_mongo(現金流量比率_Y, '現金流量比率_Y')



In [4]:
股票 = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney單日更新.xlsm', sheet_name='一日', header=4)
股票.columns = [x[8:] if len(x) >=8 else x for x in 股票.columns]
股票.rename({
    '開盤價': '還原開盤價', '最高價': '還原最高價', '最低價': '還原最低價', '收盤價': '還原收盤價',\
    '開盤價.1': '開盤價', '最高價.1': '最高價','最低價.1': '最低價', '收盤價.1': '收盤價' 
    }, axis=1, inplace=True)


In [5]:
股票['股票代號'] = 股票['股票代號'].astype(str)
日期 = 股票['日期'].iloc[0]
for name in 股票.columns[3:]:
    單資料 = dict(zip(股票['股票代號'], 股票[name]))
    單資料['日期'] = 日期
    db[name].update_one({'日期': 日期}, update={'$set': 單資料}, upsert=True)


In [4]:
# db['stock_name_mapping'].insert_many([股票名稱轉換]) # 如果要更新股票名稱再考慮這個

update_mongo(EPS_Q, 'EPS_Q')
update_mongo(預估EPS_Q, '預估EPS_Q')
update_mongo(營業收入淨額_Y, '營業收入淨額_Y')
update_mongo(殖利率, '殖利率')
update_mongo(營業收入淨額_Q, '營業收入淨額_Q')
update_mongo(EBITDA_Q, 'EBITDA_Q')

# Index 更新

In [7]:
db1 = client['Index']
#wb1.close()
#wb2.close()
wb3 = xw.Book(r"C:\Users\ChastLai\Documents\Data\每日更新DB\CMoney指數.xlsm")
func_ = wb3.macro("CM全部更新")
func_()
wb3.save()
wb3.close()
TWA00 = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney指數.xlsm', header=4, sheet_name='TWA00').iloc[:, 1:]
TWA00.rename({
    '開盤價': '還原開盤價', '最高價': '還原最高價', '最低價': '還原最低價', '收盤價': '還原收盤價',\
    '開盤價.1': '開盤價', '最高價.1': '最高價','最低價.1': '最低價', '收盤價.1': '收盤價' 
    }, axis=1, inplace=True)
VIX = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney指數.xlsm', header=4, sheet_name='VIX').iloc[:, 1:]
for i in range(len(TWA00)):
    日期 = TWA00.iloc[i]['日期']
    db1['TWA00'].update_one({'日期': 日期}, update={'$set': TWA00.iloc[i].to_dict()}, upsert=True)
for i in range(len(VIX)):
    日期 = VIX.iloc[i]['日期']
    db1['VIX'].update_one({'日期': 日期}, update={'$set': VIX.iloc[i].to_dict()}, upsert=True)

# 大量更新

import os
import pymongo
import pandas as pd
import time
import datetime
import xlwings as xw
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數

wb = xw.Book(r"C:\Users\ChastLai\Documents\Data\每日更新DB\新增資料夾\活頁簿01.xlsm")
func = wb.macro("CM全部更新")
path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\新增資料夾\\'
sheet_names = os.listdir(path)
wb1 = xw.Book()
for s in sheet_names:
    if '$' in s:
        continue
    if s in ['活頁簿01.xlsm']:
        continue
    x = '1日前'
    wb1.close()
    wb1 = xw.Book(path+s)
    name = wb1.sheets[0].name
    wb1.sheets[0].range("C1").value = wb1.sheets[0].range("C1").value.replace('6000日前', '1日前')
    wb1.sheets[0].range("C1").value = wb1.sheets[0].range("C1").value.replace('4000日前', '1日前')
    wb1.sheets[0].range("C1").value = wb1.sheets[0].range("C1").value.replace('2000日前', '1日前')
    for y in ['2000日前', '4000日前', '6000日前']:
        try:
            func()
            wb1.save()
            股票 = pd.read_excel(path + s, header=4)
            股票.dropna(how='all', axis=1, inplace=True)
            股票['股票代號'] = 股票['股票代號'].astype(str)
            # for name in ['開盤價$', '最高價$', '最低價$', '收盤價$']:
            #     if sum(股票.columns.str.contains(name)) > 0:
            #         rename = dict(zip(股票[股票.columns[股票.columns.str.contains(name)]].columns, [s[:8] + '還原' + s[8:] for s in 股票[股票.columns[股票.columns.str.contains(name)]].columns]))
            #         股票.rename(rename, axis=1, inplace=True)
            # for name in ['開盤價.1', '最高價.1', '最低價.1', '收盤價.1']:
            #     if sum(股票.columns.str.contains(name)) > 0:
            #         rename = dict(zip(股票[股票.columns[股票.columns.str.contains(name)]].columns, [s[:8] + s[8:-2] for s in 股票[股票.columns[股票.columns.str.contains(name)]].columns]))
            #         股票.rename(rename, axis=1, inplace=True)
            print(f'{name} 更新...')
            日期 = [x[:8] for x in 股票.columns[2:]]
            df = 股票.drop('股票名稱', axis=1).set_index('股票代號').T
            df.index = pd.to_datetime(日期)
            df['日期'] = df.index
            df.sort_index(inplace=True)
            db[name].insert_many(df.to_dict('records'))
            df_ = pd.DataFrame(db[name].find({}, {'_id': 0}))
            df_.dropna(subset='日期', inplace=True)
            df_.drop_duplicates(subset='日期', keep='last', inplace=True)
            df_.sort_values('日期', inplace=True)
            db[name].delete_many({})
            db[name].insert_many(df_.to_dict('records'))
            gc.collect()
            if x in wb1.sheets[0].range("C1").value:
                wb1.sheets[0].range("C1").value = wb1.sheets[0].range("C1").value.replace(x, y)
                x = y
                func()
                wb1.save()
            else:
                break
        except Exception as e:
            print(s, e)
    # for name in 股票.columns[2:]:
    #     日期 = pd.to_datetime(name[:8])
    #     單資料 = dict(zip(股票['股票代號'], 股票[name]))
    #     單資料['日期'] = 日期
    #     if '還原' in s:
    #         db_name = '還原' + name[8:]
    #     else:
    #         db_name = name[8:]

    #     db[name[8:]].update_one({'日期': 日期}, update={'$set': 單資料}, upsert=True)
    

# 舊DB轉換
db_old = client['Data']
field_list = db_old['2330'].find_one({}, {'_id': 0}).keys()

for field in field_list:
    df = pd.concat([pd.DataFrame(db_old[x].find({}, {'_id': 0, '日期': 1, field: 1})).set_index('日期') for x in db_old.list_collection_names()], axis=1).dropna(how='all')
    df.columns = db_old.list_collection_names()
    df = df.sort_index()
    df['日期'] = df.index
    db[field].insert_many(df.to_dict('records'))
